# Preprocess IMU

## Modules

In [38]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import dbpd
import numpy as np
import os
import tsdf

from dbpd import DataColumns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constants

In [23]:
sensor = 'IMU'

path_to_data = '../../../tests/data/'
input_path = os.path.join(path_to_data, '1.sensor_data')
output_path = os.path.join(path_to_data, '2.preprocessed_data')

meta_filename = f'{sensor}_meta.json'
values_filename = f'{sensor}_samples.bin'
time_filename = f'{sensor}_time.bin'
quality_filename = f'{sensor}_quality.bin'

l_acceleration_cols = [
    DataColumns.ACCELERATION_X,
    DataColumns.ACCELERATION_Y,
    DataColumns.ACCELERATION_Z
]
l_gyroscope_cols = [
    DataColumns.ROTATION_X,
    DataColumns.ROTATION_Y,
    DataColumns.ROTATION_Z,
]

rotation_units = 'deg/s'
acceleration_units = 'm/s^2'

# filtering
fs = 100
lower_cutoff_frequency = 0.3
filter_order = 4

# store metadata
d_channels = {
    DataColumns.ROTATION_X: rotation_units,
    DataColumns.ROTATION_Y: rotation_units,
    DataColumns.ROTATION_Z: rotation_units,
    DataColumns.ACCELERATION_X: acceleration_units,
    DataColumns.ACCELERATION_Y: acceleration_units,
    DataColumns.ACCELERATION_Z: acceleration_units,
    f'grav_{DataColumns.ACCELERATION_X}': acceleration_units,
    f'grav_{DataColumns.ACCELERATION_Y}': acceleration_units,
    f'grav_{DataColumns.ACCELERATION_Z}': acceleration_units,
}

## Load data

In [49]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(input_path, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.sample(2)

,time,acceleration_x,acceleration_y,acceleration_z,rotation_x,rotation_y,rotation_z
5109,10.040039,1261,-168,-1673,-774,-59,-255
63936,10.000000,-2130,807,-559,-2878,216,75


## Preprocess data

In [50]:
ppp = dbpd.PreprocessingPipelineConfig(
    time_column='time',
    sampling_frequency=fs,
    resampling_frequency=fs,
    gyroscope_units=rotation_units
    )

df['time'] = dbpd.imu_preprocessing.transform_time_array(
    time_array=df['time'],
    scale_factor=1000, 
    data_in_delta_time=True)

df = dbpd.imu_preprocessing.resample_data(
    config=ppp,
    time_abs_array=np.array(df['time']),
    values_unscaled=np.array(df[l_acceleration_cols + l_gyroscope_cols]),
    scale_factors=metadata_samples.scale_factors)

for col in l_acceleration_cols:
    for result, side_pass in zip(['filt', 'grav'], ['lp', 'hp']):
        df[f'{result}_{col}'] = dbpd.imu_preprocessing.butterworth_filter(
            config=ppp,
            single_sensor_col=np.array(df[col]),
            order=filter_order,
            cutoff_frequency=lower_cutoff_frequency,
            passband=side_pass
            )
        
    df = df.drop(columns=[col])
    df = df.rename(columns={f'filt_{col}': col})

# Store data

In [52]:
# change channels and units accordingly
metadata_samples.__setattr__('channels', list(d_channels.keys()))
metadata_samples.__setattr__('units', list(d_channels.values()))

# change output path
metadata_samples.__setattr__('file_dir_path', output_path)  
metadata_time.__setattr__('file_dir_path', output_path)

In [53]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

# store binaries and metadata
tsdf.write_dataframe_to_binaries(output_path, df, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], meta_filename)